# Chapter 7. 앙상블 학습과 랜덤 포레스트
일련의 예측기를 앙상블이라고 부름. 결정 트리의 앙상블은 랜덤 포레스트.

## 7.1 투표 기반 분류기
각 분류기의 예측을 모아 가장 많이 선택된 클래스 예측. 다수결 투표로 정해지는 분류기를 직접 투표 분류기라고 함.각 분류기가 약한 학습기라도 충분히 많고 다양하면 앙상블은 강한 학습기가 될 수 있음.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

#5장에서 소개한 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 1.0
SVC 0.9
VotingClassifier 0.9


개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스 예측: 간접 투표. 확률이 높은 투표에 비중을 두어 직접 투표 방식보다 성능이 높음. voting="soft"로 바꾸고 모든 분류기가 클래스의 확률 추정하면 사용 가능

## 7.2 배깅과 페이스팅
같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성해 분류기를 각각 다르게 학습시키는 것. 훈련 세트에서 중복을 허용하여 샘플링하는 방식=배깅. 중복을 허용하지 않고 샘플링하는 방식=페이스팅. 수집 함수는 분류일 때 통계적 최빈값, 회귀 때 평균 값 계산

### 7.2.1 사이킷런의 배깅과 페이스팅
BaggingClassifier

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)

### 7.2.2 oob 평가
BaggingClassifier는 기본적으로 중복을 허용해 훈련세트 크기만큼 샘플 선택. 훈련되지 않은 훈련 샘플을 oob(out of bag) 샘플이라 부름.

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9375

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

In [7]:
bag_clf.oob_decision_function_

array([[0.07653061, 0.92346939],
       [0.95789474, 0.04210526],
       [0.24468085, 0.75531915],
       [0.03448276, 0.96551724],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.94413408, 0.05586592],
       [0.91666667, 0.08333333],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.02366864, 0.97633136],
       [0.44508671, 0.55491329],
       [1.        , 0.        ],
       [0.9558011 , 0.0441989 ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.17525773, 0.82474227],
       [0.76923077, 0.23076923],
       [1.        , 0.        ],
       [0.23295455, 0.76704545],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.98360656, 0.01639344],
       [0.29310345, 0.70689655],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.

## 7.3 랜덤 패치와 랜덤 서브스페이스
max_features, bootstrap_features는 특성에 대한 샘플링. 훈련 특성과 샘플을 모두 샘플링하는 것을 랜덤 패치 방법이라 함. 훈련 샘플은 모두 사용하고 특성은 샘플링하는 것은 랜덤 서브스페이스 방식.

## 7.4 랜덤 포레스트
배깅 방법을 적용한 결정 트리 앙상블. max_samples를 훈련 세트 크기로 지정

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

### 7.4.1 엑스트라 트리
극단적으로 무작위한 트리의 랜덤 포레스트: 익스트림 랜덤 트리 앙상블. 편향이 늘어나지만 분산을 낮춤. 속도 빠름

### 7.4.2 특성 중요도
어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인해 특성의 중요도 측정. 가중치 평균이며 각 노드의 가중치는 연관된 훈련 샘플 수와 같음

In [10]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09318333614189511
sepal width (cm) 0.023420865082202657
petal length (cm) 0.43052487742423545
petal width (cm) 0.45287092135166673


## 7.5 부스팅
부스팅은 약한 학습기를 여러 개 연결해 강한 학습기를 만드는 앙상블 방법. 앞의 모델을 보완해나가며 일련의 예측기 학습.

### 7.5.1 에이다부스트
이전 모델이 과소적합 했던 훈련 샘플의 가중치를 높이기. 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰짐. 첫 번째 분류기를 훈련시키고 예측을 만든 후 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높임. 두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 다시 예측을 만듦.

In [11]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그래디언트 부스팅
앙상블 이전까지 오차를 보정하도록 예측기 순차적으로 추가. 이전 예측기가 만든 잔여 오차에 새로운 예측기 학습. 그래디언트 트리 부스팅 or 그래디언트 부스티드 회귀 트리

In [14]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [15]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [16]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [19]:
# 테스트셋 생성
X_new = np.array([[0.8]])
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [20]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [22]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up=0
for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error <min_val_error:
    min_val_error = val_error
    error_going_up=0
  else:
    error_going_up+=1
    if error_going_up ==5:
      break # 조기 종료

In [23]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [24]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.39648
[1]	validation_0-rmse:0.33187
[2]	validation_0-rmse:0.29786
[3]	validation_0-rmse:0.28276
[4]	validation_0-rmse:0.27787
[5]	validation_0-rmse:0.27783
[6]	validation_0-rmse:0.27584
[7]	validation_0-rmse:0.27477
[8]	validation_0-rmse:0.27478
[9]	validation_0-rmse:0.27460
[10]	validation_0-rmse:0.27545
[11]	validation_0-rmse:0.27586


## 7.6 스태킹
앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델 훈련